# SETUP

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine

# GET DATA

In [18]:
# fetch a web page
htlm_page = requests.get("https://www.francetvinfo.fr/politique/")
print(htlm_page.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Politique</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Politique" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Politique&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_FR" />

In [19]:
page = BeautifulSoup(htlm_page.text, "html5lib")
print(page.get_text()[:1000])

Politique  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/politique/"+queryString+myHash));  (function()

## EXTRACT TITLES

In [20]:
# extract title
summaries = page.find_all('div', class_="flowItem")

In [21]:
summaries[0]

<div class="flowItem feature" entity="CMS-Video_3848761"><a href="/politique/parti-communiste-francais/municipales-saint-denis-un-fief-du-pcf-menace_3848761.html"><figure id="image_3645881"><span class="videoIcon"></span>    <img alt="Laurent Russier (PCF)" botag="CMS-IMAGE_3645881" class="b-lazy" data-src="/image/75rw29mdb-d00b/840/472/21056725.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="Laurent Russier (PCF) | FRANCEINFO" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">Municipales : Saint-Denis, un fief du PCF menacé</h2><div class="description" field="description">Saint-Denis (Seine-Saint-Denis) est la plus grande ville dirigée par un maire communiste. Le PCF tient cette ville de 110 000 habitants depuis la fin de la Seconde Guerre mondiale.</div></div></a></div>

In [22]:
summaries[0].select_one("h2").get_text()

'Municipales : Saint-Denis, un fief du PCF menacé'

In [23]:
len(summaries)

28

In [25]:
politic_news = [x.select_one("h2").get_text() for x in summaries]

In [26]:
politic_news

['Municipales : Saint-Denis, un fief du PCF menacé',
 'Coronavirus, 49.3, Adèle Haenel aux César... Le "8h30 franceinfo" de Sibeth Ndiaye',
 "DIRECT. Penelope Fillon était-elle réellement l'assistante parlementaire de son mari ? Suivez la quatrième journée de leur procès",
 'Réforme des retraites : "Le recours au 49.3 est un scandale", estime Ian Brossat (PCF)',
 "Réforme des retraites : malgré le recours au 49.3, les opposants n'ont pas abdiqué",
 'VIDEO. Julien Bayou et Damien Abad réagissent au recours au 49.3 du Premier ministre',
 '"Nous débloquerons ce qu\'il faudra" pour les entreprises pénalisées par le coronavirus, assure Bruno Le Maire',
 'VIDEO. Réforme des retraites : Jean-Luc Mélenchon appelle à des mobilisations populaires "partout dans le pays"',
 'VIDEO. Recours au 49.3 sur la réforme des retraites : face au "cynisme" du gouvernement, les députés LR veulent présenter un "contre-projet"',
 'Municipales 2020 à Bordeaux : tout comprendre des enjeux du scrutin, surtout si v

# WRITE THE DATA IN POSTGRESQL DB

In [27]:
df = pd.DataFrame({"source": "franceinfo", "headline": politic_news, "date": pd.to_datetime('today').to_pydatetime().date()})

In [28]:
df.head()

,source,headline,date
0,franceinfo,"Municipales : Saint-Denis, un fief du PCF menacé",2020-03-02
1,franceinfo,"Coronavirus, 49.3, Adèle Haenel aux César... L...",2020-03-02
2,franceinfo,DIRECT. Penelope Fillon était-elle réellement ...,2020-03-02
3,franceinfo,"Réforme des retraites : ""Le recours au 49.3 es...",2020-03-02
4,franceinfo,Réforme des retraites : malgré le recours au 4...,2020-03-02


In [29]:
# create an engine
alchemy_engine = create_engine('postgresql+psycopg2://antoinemertz@localhost/news', pool_recycle=3600)

In [30]:
conn = alchemy_engine.connect()
table = "politic"

In [31]:
try:
    frame = df.to_sql(name=table, con=conn, if_exists='append', index=False)
except Exception as ex:
    print(ex)
conn.close()